In [88]:
# imports
import pandas as pd 
import numpy as np
from urllib3 import request as uReq
import json

#Requsting content from open url with GET method
get_json=uReq('GET','https://data.nasa.gov/resource/y77d-th95.json')
json_data= get_json.json()      

with open('json_data.json',"w+")as json_file:
    json.dump(json_data, json_file)                             # Creating json file and dumpingconte extracted content from it

structured_dataset=pd.json_normalize(json_data)    # structured_dataset.to_excel('structured_dataset.xlsx')          

old_column= ['name', 'id', 'nametype', 'recclass', 'mass', 'year', 'reclat',
       'reclong', 'geolocation.coordinates']
new_column= ['Name','Id','Nametype','Recclass','Mass','Year','Reclat','Reclong','Pointer Coordinates']
structured_dataset.columns=structured_dataset.columns.map(dict(zip(old_column,new_column)))

structured_dataset.drop(np.nan,axis=1,inplace=True)

structured_dataset['Mass']= structured_dataset.Mass.astype(float)
structured_dataset['Id']= structured_dataset.Id.astype(int)
structured_dataset['Year']=structured_dataset['Year'].apply(lambda x: int(x[:4]) if isinstance(x, str) else 0)
structured_dataset['Reclat']=structured_dataset['Reclat'].astype(float)
structured_dataset['Reclong']=structured_dataset['Reclong'].astype(float) 

structured_dataset.to_excel("structured_dataset.xlsx")  # saving structured format data into excel file  this might require to install 'openpyxl' library